In [1]:
import importlib
import sys
import torch
import pickle
import os

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')


from model.dropout_uncertainty_enc_dec_LSTM.dropout_uncertainty_model import DropoutUncertaintyEncoderDecoderLSTM

In [2]:
#load model
file_path_model = '../../../training_variational_dropout/PCR-Test/PCR_full_grad_norm_new_4layer.pkl'
model = DropoutUncertaintyEncoderDecoderLSTM.load(file_path_model, dropout=0.1)

# Load the dataset
file_path_data_set = '../../../../../encoded_data/PCR_5_test.pkl'
pcr_test_dataset = torch.load(file_path_data_set, weights_only=False)

Data set categories:  ([('concept:name', 10, {'Callback timeout': 1, 'EOS': 2, 'Export result': 3, 'Export to EMS': 4, 'Match patient data': 5, 'Receive sample state': 6, 'Send notification': 7, 'Wait for plate validation': 8, 'timeout': 9}), ('cpee:lifecycle:transition', 6, {'EOS': 1, 'activity/calling': 2, 'activity/done': 3, 'activity/receiving': 4, 'dataelements/change': 5})], [('case_elapsed_time', 1, {}), ('event_elapsed_time', 1, {}), ('day_in_week', 1, {}), ('seconds_in_day', 1, {})])
Encoder input features:  [['concept:name', 'cpee:lifecycle:transition'], ['case_elapsed_time', 'event_elapsed_time', 'day_in_week', 'seconds_in_day']]
Decoder input+output features:  [['concept:name'], ['case_elapsed_time', 'event_elapsed_time']]


Sequence length of decoder output:  4


Cells hidden size:  128
Number of LSTM layer:  4
Dropout rate:  0.1


Encoder number of labels for each input feature (categorical, numerical):  [[10, 6], [1, 1, 1, 1]]
Encoder indices of tensors in dataset used a

In [3]:
import evaluation.probabilistic_evaluation
importlib.reload(evaluation.probabilistic_evaluation)
from evaluation.probabilistic_evaluation import ProbabilisticEvaluation

new_eval = ProbabilisticEvaluation(model, 
                                   pcr_test_dataset,
                                   concept_name='concept:name',
                                   num_processes=16,
                                   #growing_num_values = [],
                                   growing_num_values = ['case_elapsed_time'],
                                   samples_per_case = 1000,
                                   sample_argmax = False,
                                   use_variance_cat = True,
                                   use_variance_num = True,
                                   all_cat=['concept:name'],
                                   all_num=['case_elapsed_time', 'event_elapsed_time']
                                   )


In [4]:
def save_chunk(results, i):
    chunk_number = (i + 1)
    filename = os.path.join(output_dir, f'results_part_{chunk_number:03d}.pkl')
    with open(filename, 'wb') as f:
        pickle.dump(results, f)
    print(f"Saved {len(results)} results to {filename}")
    
# Directory path on NAS - evaluation results directory on top of the project
  
output_dir = '../../../../../../../evaluation_results/PCR/gn_l4/'  

save_every = 50

results = {}
for i, (case_name, prefix_len, prefix, predicted_suffixes, suffix, mean_prediction) in enumerate(new_eval.evaluate_multi_processing(random_order=False)):
    # print(case_name, prefix_len)
    assert((case_name, prefix_len) not in results)
    results[(case_name, prefix_len)] = (prefix, suffix, mean_prediction, predicted_suffixes)
    # print(prefix_len, len(suffix))
    if (i + 1) % save_every == 0:
        save_chunk(results, i)
        results = {}

if len(results):
    save_chunk(results, i)

  0%|          | 0/998 [00:00<?, ?it/s]

Saved 50 results to ../../../../../../../evaluation_results/PCR/gn_l4/results_part_050.pkl


Saved 50 results to ../../../../../../../evaluation_results/PCR/gn_l4/results_part_100.pkl


Saved 50 results to ../../../../../../../evaluation_results/PCR/gn_l4/results_part_150.pkl


Saved 50 results to ../../../../../../../evaluation_results/PCR/gn_l4/results_part_200.pkl


...

Saved 50 results to ../../../../../../../evaluation_results/PCR/gn_l4/results_part_18100.pkl


Saved 50 results to ../../../../../../../evaluation_results/PCR/gn_l4/results_part_18150.pkl


Saved 50 results to ../../../../../../../evaluation_results/PCR/gn_l4/results_part_18200.pkl


Saved 50 results to ../../../../../../../evaluation_results/PCR/gn_l4/results_part_18250.pkl


Saved 7 results to ../../../../../../../evaluation_results/PCR/gn_l4/results_part_18257.pkl
